Use Case 9
==========

Problem Definition:
-------------------

A climate scientist wishes to analyse potential correlations between *Ozone* and *Cloud* ECVs.

Required Toolbox Features:
--------------------------

* Access to and ingestion of ESA CCI Ozone and Cloud data (Atmosphere Mole Content of Ozone and Cloud Cover)
* Geometric adjustments (coregistration)
* Spatial (point, polygon) and temporal subsetting
* Visualisation of time series
* Correlation analysis, scatter-plot of correlation statistics, saving of image and correlation statistics on disk (format options)

Ingest data and create datasets
-------------------------------

In [1]:
from cate.core.ds import DATA_STORE_REGISTRY
import cate.ops as ops
from cate.util import ConsoleMonitor

In [2]:
monitor = ConsoleMonitor()

In [3]:
data_store = DATA_STORE_REGISTRY.get_data_store('esa_cci_odp')
local_store = DATA_STORE_REGISTRY.get_data_store('local')

In [4]:
oz_remote_sources = data_store.query('esacci.OZONE.mon.L3.NP.multi-sensor.multi-platform.MERGED.fv0002.r1')
oz_remote_sources

[esacci.OZONE.mon.L3.NP.multi-sensor.multi-platform.MERGED.fv0002.r1,
 esacci.OZONE.day.L3S.TC.SCIAMACHY.Envisat.MERGED.fv0100.r1,
 esacci.OZONE.day.L3S.TC.GOME-2.Metop-A.MERGED.fv0100.r1,
 esacci.OZONE.day.L3S.TC.GOME.ERS-2.MERGED.fv0100.r1,
 esacci.OZONE.day.L3S.TC.multi-sensor.multi-platform.MERGED.fv0100.r1]

In [7]:
%pdoc oz_remote_sources[0].make_local

Object `oz_remote_sources[0].make_local` not found.


In [ ]:
oz_remote_sources[0].make_local(local_name='ozone.mon',
                                time_range='2007-01-01, 2007-03-31',
                                monitor=monitor)

In [ ]:
cl_remote_sources = data_store.query('esacci.CLOUD.mon.L3C.CLD_PRODUCTS.multi-sensor.multi-platform.ATSR2-AATSR.2-0.r1')
cl_remote_sources

In [ ]:
cl_remote_sources[0].make_local(local_name='clouds1.mon',
                                time_range='2007-01-01, 2007-03-31',
                                monitor=monitor)

In [ ]:
local_store

In [ ]:
cc = ops.open_dataset('local.clouds1.mon')

In [ ]:
cc

In [ ]:
oz = ops.open_dataset('local.ozone.mon')

In [ ]:
oz

Filter datasets to select the desired variables
-----------------------------------------------

In [ ]:
cc_tot = ops.select_var(cc, 'cfc')
oz_tot = ops.select_var(oz, 'O3_du_tot')

In [ ]:
oz_tot

Plot the first time slice of the dataset

In [ ]:
%matplotlib inline
ops.plot_map(oz_tot, var='O3_du_tot', time='2007-01-01', file='/home/ccitbx/Desktop/fig1.png')

In [ ]:
ops.plot_map(cc_tot, var='cfc', time='2007-01-01', file='/home/ccitbx/Desktop/fig2.png')

Co-register datasets by resampling
----------------------------------

In [ ]:
print(cc_tot['cfc'].shape)
print(oz_tot['O3_du_tot'].shape)

In [ ]:
cc_tot_res = ops.coregister(oz_tot, cc_tot)

In [ ]:
print(cc_tot_res['cfc'].shape)
print(oz_tot['O3_du_tot'].shape)

In [ ]:
ops.plot_map(cc_tot_res, var='cfc', time='2007-01', file='/home/ccitbx/Desktop/fig3.png')

Select the desired spatial region
---------------------------------

In [ ]:
africa = '-20.0, -40.0, 60.0, 40.0'
# 'lon_min, lat_min, lon_max, lat_max'
cc_tot_africa = ops.subset_spatial(cc_tot_res, africa)
oz_tot_africa = ops.subset_spatial(oz_tot, africa)

In [ ]:
ops.plot_map(cc_tot_africa, var='cfc', time='2007-01-01', file='/home/ccitbx/Desktop/fig4.png')

In [ ]:
ops.plot_map(cc_tot_africa, var='cfc', time='2007-01-01', 
                 region=africa, file='/home/ccitbx/Desktop/fig5.png')

Select the desired temporal region
----------------------------------

In [ ]:
cc_tot_janfeb = ops.subset_temporal(cc_tot_africa, '2007-01-01, 2007-02-28')
oz_tot_janfeb = ops.subset_temporal(oz_tot_africa, '2007-01-01, 2007-02-28')

In [ ]:
print(cc_tot_janfeb.time)

In [ ]:
print(oz_tot_janfeb.time)

Retrieve and plot timeseries
----------------------------

In [ ]:
oz_ts_point = ops.tseries_point(oz_tot_janfeb, point='50, 50')
cc_ts_point = ops.tseries_point(cc_tot_janfeb, point='50, 50')
oz_ts_mean = ops.tseries_mean(oz_tot_janfeb, var='O3_du_tot')
cc_ts_mean = ops.tseries_mean(cc_tot_janfeb, var='cfc')

In [ ]:
print(oz_ts_mean)

In [ ]:
print(cc_ts_mean)

In [ ]:
ops.plot(cc_ts_mean, 'cfc', file='/home/ccitbx/Desktop/fig6.png')

In [ ]:
ops.plot(oz_ts_mean, 'O3_du_tot', file='/home/ccitbx/Desktop/fig7.png')

Do correlation analysis
-----------------------

In [ ]:
correlation = ops.pearson_correlation(cc_ts_mean, oz_ts_mean, 'cfc', 'O3_du_tot',
                                          file="/home/ccitbx/Desktop/correlation.txt")
correlation

In [ ]:
correlation = ops.pearson_correlation(cc_tot_janfeb, oz_tot_janfeb, 'cfc', 'O3_du_tot')

In [ ]:
correlation

In [ ]:
ops.plot_map(correlation, var='corr_coef', 
             region=africa, file='/home/ccitbx/Desktop/fig8.png')